<center><h1>Telco Churn Project</h1></center>

# Table of contents <a name ='toc'></a>
1. [Project Summary](#project_summary)
    1. [Project Objectives](#project_objectives)
2. [Data Acqisition](#data_acquisition)
5. [Preparation](#preparation)
6. [Exploratory Data Analysis](#exp_data_analysis)
2. [Drivers of Churn](#drivers_of_churn)
3. [Machine Learning Construction](#ml_construction)
7. [Statistical Testing](#stat_testing)
8. [Modeling](#modeling)
9. [Model Evaluation](#model_eval)


<hr style="border-top: 10px groove grey; margin-top: 1px; margin-bottom: 1px"></hr>

## Project Summary <a name="project_summary"></a>
[Table of Contents](#toc)

> - Create a Jupyter Notebook Report that shows processes and analysis with the goal of finding drivers for customer churn.
> - Within README.md file, include project description with goals, inital hypotheses, a data dictonary, project planning, instructions on how to recreate your project, answers to hypotheses, key findings, recommendations, and takeaways from the project.
> - CSV file with customer_id, probabilty of churn, and prediction of churn (1 = churn, 0 = not_churn). These predictions will be dervied from the best performing model on the test portion of the data. 
> - Any abstracted modules that are created to make the presentation more clean, during the acquistion and preparation of data.
> - Notebook walkthrough and presentation with a high-level overview of the entire project.

<hr style="border-top: 10px groove tan; margin-top: 1px; margin-bottom: 1px"></hr>


### Drivers of Churn <a name="drivers_of_churn"></a>


<hr style="border-top: 10px groove tan; margin-top: 1px; margin-bottom: 1px"></hr>


### Machine Learning Construction <a name="ml_construction"></a>

